In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import itertools
import copy
import random
import matplotlib.pyplot as plt

# Set up

## Short description##

BSc thesis description:

The title of the thesis was 'The Empirical Inefficiency of Equilibria in Coordination games on Social Networks'. 
 In the thesis I explore a case of algorithmic game theory on Facebook and model-based networks. 
 For more information regarding the theory and methodology, please contact me directly.The data from real-world Facebook networks was collected via the Netvizz application version in the autumn of 2014


### Data pre-processing to a graph

In [14]:
fbgraph=open('somenetwork.gdf','r').read().split('\n')

In [6]:
#split to two lists: nodes and edges

nodes=[]

for node in fbgraph:
    if node!='edgedef>node1 VARCHAR,node2 VARCHAR':
        nodes.append(node)
    else:
        break
        
edges=[i for i in fbgraph if i not in nodes]
edges = edges[1:]

In [7]:
#store nodeIDs (type string) only 

nodeIDs=[]
for i in nodes:
    nodeID=i.split(',',1)[0]
    nodeIDs.append(nodeID)
nodeIDs = nodeIDs[1:]

In [8]:
#make dictionary with only keys (nodeIDs)

grdict={x:[] for x in nodeIDs}

In [9]:
#Add edges between nodeIDs (keys) and their neighbors (values)

for i in grdict.keys():
    for j in edges: 
        if i==j.split(',',1)[0]:
            grdict[i].append(j.split(',',1)[1])
            grdict[j.split(',',1)[1]].append(i)

In [10]:
# FOR CLARITY - give nodes new names: integers

new_names={}
for i,j in enumerate(grdict.keys()):
    new_names[j]=i

In [11]:
#change node IDs in dictionary values to integer values

for e, k in enumerate(grdict.values()):
    #for every node in neighbor list
    for e2, i in enumerate(k):
        for j in new_names.keys():
            if i==j:
                grdict.values()[e][e2]=new_names[j]

In [12]:
#change node IDs in dictionary keys

for i in grdict.keys():
    for j in new_names.keys():
        if i==j:
            grdict[new_names[j]] = grdict.pop(i)

### EXPERIMENTS###

In [110]:
sizes = [1,3,4,5,6,7,8,9,10,2] # sizes ok K set

#already did with 2

# |K| = 1

In [111]:
### EXPERIMENT K size = 1

iteration = 1
# for j in xrange(len(sizes)): #for different sizes of k
    # initialize dataframe k | k size | PoA
data = pd.DataFrame(columns=('k', 'k size', 'PoA'))
    ### 1

    #random K set
    #k = random.sample(xrange(len(nodeIDs)), sizes[j])

for i in xrange(200):
    ### 1

    #random K set
    k = random.sample(xrange(len(nodeIDs)), sizes[0])

    ### 2

    #ADD ALL CALCULATIONS FOR THIS k
    grd = grdict.copy()
    gdc = copy.copy(grd)
    # make dictionary into graph
    grd = nx.to_networkx_graph(grd)

    ############## START STRATEGY DISTRIBUTION ###########

    ### Initialize Identity matrix - everyone has their own strategy

    strategies = np.identity(len(nodeIDs))

    ### Make a copy of real graph to find strategy distribution (because we'll need to delete some edges)

    H = grd.copy()

    ### For strategy distribution disregard edges between nodes in k

    #all possible edge combinations between the k_nodes
    knode_pedges=list(itertools.combinations(k, 2))

    #Identify if the nodes have edges among each other and store them
    edges_between_k=[]
    for pedge in knode_pedges:
        for edge in H.edges():
            if set(pedge)==set(edge):
                edges_between_k.append(pedge)

    # For the strategy distribution delete the edges between k in a duplicate H of our real graph grd! 
    H.remove_edges_from(edges_between_k)

    ### Find neighborhood of range = 2 for nodes in k and assign them the strategy of that node

    #Check how many nodes have neighborhood of 2 less than a 100

    for i in xrange(len(k)):
        for j in nx.ego_graph(H, k[i], 2).nodes():
            if j not in k:
                strategies[j][k[i]]=1

    np.where(strategies[4] == 1)[0].tolist()

    ### STRATEGY ASSIGNMENT TO NODES

    #to avoid confusion attribute will be called plays rather than strategies (to separate from strategy matrix)

    for i in xrange(len(nodeIDs)):
        key="plays"
        #step of assigining attributes
        grd.node[i].setdefault(key, np.where(strategies[i] == 1)[0].tolist()) 

    gego = grd.copy() # duplicate graph for 'OPTIMAL' calculations later

    ### Create ATTRIBUTES in graph: neighbors, current_play, possible_play (to find what is my payoff 
    #                                                                       if I play this strategy)

    for i in xrange(len(nodeIDs)):
        # step of assigining attributes
        grd.node[i].setdefault("neighbors", gdc[i])
        # everybody starts at their own strategy; 'possible_play' attribute for strategy update
        grd.node[i].setdefault("current_play", i)
        grd.node[i].setdefault("possible_play", i)

    ### UPDATE STRATEGIES ### 

    # the idea is to have two strategy lists that will be compared after every iteration through all the nodes
    # when the lists are identical, no player will switch strategies -> hence, Nash equilibrium


    # Initialize two lists to compare 
    past_strategies = []
    new_strategies = []
    for i in xrange(len(nodeIDs)):
        new_strategies.append(grd.node[i]['current_play'])
        past_strategies.append(0)


    ### WHILE LOOP TO FIND BEST STRATEGIES ###


    i = 0
    while past_strategies != new_strategies:
        past_strategies = copy.copy(new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(grd.node[inode]['plays'])):
                grd.node[inode]['possible_play'] = grd.node[inode]['plays'][strategy]
                for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                    if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                grd.node[inode]['current_play'] = grd.node[inode]['plays'][random_index]
                new_strategies[inode] = grd.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        ind_payoff = 0
        for neighbor in xrange(len(grd.node[inode]['neighbors'])):
            if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                ind_payoff += 1
        ind_payoffs.append(ind_payoff)

    social_welfare = sum(ind_payoffs)

    ### OPTIMAL WELFARE APPROXIMATION ###

    source_neighborhoods = []

    # choose source with biggest ego_network
    for source in xrange(len(k)):
        source_neighborhoods.append(len(nx.ego_graph(gego, k[source], 2).nodes()))

    for node in xrange(len(nodeIDs)):
        gego.node[node]["neighbors"] = gdc[node]
        if node in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
            # every node in ego graph gets only one strategy (source's strategy)
            gego.node[node]["current_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["possible_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["plays"] = [k[np.argmax(source_neighborhoods)]]
        elif node not in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
        #    # prepare for best response
            gego.node[node]["current_play"] = node
            gego.node[node]["possible_play"] = node

    opt_past_strategies = [] # Initialize two lists to compare 
    opt_new_strategies = []
    for i in xrange(len(nodeIDs)):
        opt_new_strategies.append(gego.node[i]['current_play'])
        opt_past_strategies.append(0)


    ### While loop to find best strategies ###


    while opt_past_strategies != opt_new_strategies:
        opt_past_strategies = copy.copy(opt_new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(gego.node[inode]['plays'])):
                gego.node[inode]['possible_play'] = gego.node[inode]['plays'][strategy]
                for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                    if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                gego.node[inode]['current_play'] = gego.node[inode]['plays'][random_index]
                opt_new_strategies[inode] = gego.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    opt_ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        opt_ind_payoff = 0
        for neighbor in xrange(len(gego.node[inode]['neighbors'])):
            if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                opt_ind_payoff += 1
        opt_ind_payoffs.append(opt_ind_payoff)

    opt_social_welfare = sum(opt_ind_payoffs)

    if float(social_welfare) != 0:
        PoA = float(opt_social_welfare) / float(social_welfare)
    elif float(social_welfare) == 0:
        PoA = float('inf')

    ### 3
    # add data to dataframe k | k size | PoA
    df2 = pd.DataFrame([[k, len(k), social_welfare, opt_social_welfare, PoA ]], columns=('k', 'k size', 'SW', 'reasonable SW', 'PoA'))
    frames = [data, df2]
    data = pd.concat(frames)
    iteration += 1
    print iteration
data.to_csv('FB4-random-k%s.csv' %str(len(k)), encoding='utf-8')


# |K| = 3 #

In [113]:
### EXPERIMENT K size = 3

# for j in xrange(len(sizes)): #for different sizes of k
    # initialize dataframe k | k size | PoA
data = pd.DataFrame(columns=('k', 'k size', 'PoA'))
    ### 1

    #random K set
    #k = random.sample(xrange(len(nodeIDs)), sizes[j])

for i in xrange(200):
    ### 1

    #random K set
    k = random.sample(xrange(len(nodeIDs)), sizes[1])

    ### 2

    #ADD ALL CALCULATIONS FOR THIS k
    grd = grdict.copy()
    gdc = copy.copy(grd)
    # make dictionary into graph
    grd = nx.to_networkx_graph(grd)

    ############## START STRATEGY DISTRIBUTION ###########

    ### Initialize Identity matrix - everyone has their own strategy

    strategies = np.identity(len(nodeIDs))

    ### Make a copy of real graph to find strategy distribution (because we'll need to delete some edges)

    H = grd.copy()

    ### For strategy distribution disregard edges between nodes in k

    #all possible edge combinations between the k_nodes
    knode_pedges=list(itertools.combinations(k, 2))

    #Identify if the nodes have edges among each other and store them
    edges_between_k=[]
    for pedge in knode_pedges:
        for edge in H.edges():
            if set(pedge)==set(edge):
                edges_between_k.append(pedge)

    # For the strategy distribution delete the edges between k in a duplicate H of our real graph grd! 
    H.remove_edges_from(edges_between_k)

    ### Find neighborhood of range = 2 for nodes in k and assign them the strategy of that node

    #Check how many nodes have neighborhood of 2 less than a 100

    for i in xrange(len(k)):
        for j in nx.ego_graph(H, k[i], 2).nodes():
            if j not in k:
                strategies[j][k[i]]=1

    np.where(strategies[4] == 1)[0].tolist()

    ### STRATEGY ASSIGNMENT TO NODES

    #to avoid confusion attribute will be called plays rather than strategies (to separate from strategy matrix)

    for i in xrange(len(nodeIDs)):
        key="plays"
        #step of assigining attributes
        grd.node[i].setdefault(key, np.where(strategies[i] == 1)[0].tolist()) 

    gego = grd.copy() # duplicate graph for 'OPTIMAL' calculations later

    ### Create ATTRIBUTES in graph: neighbors, current_play, possible_play (to find what is my payoff 
    #                                                                       if I play this strategy)

    for i in xrange(len(nodeIDs)):
        # step of assigining attributes
        grd.node[i].setdefault("neighbors", gdc[i])
        # everybody starts at their own strategy; 'possible_play' attribute for strategy update
        grd.node[i].setdefault("current_play", i)
        grd.node[i].setdefault("possible_play", i)

    ### UPDATE STRATEGIES ### 

    # the idea is to have two strategy lists that will be compared after every iteration through all the nodes
    # when the lists are identical, no player will switch strategies -> hence, Nash equilibrium


    # Initialize two lists to compare 
    past_strategies = []
    new_strategies = []
    for i in xrange(len(nodeIDs)):
        new_strategies.append(grd.node[i]['current_play'])
        past_strategies.append(0)


    ### WHILE LOOP TO FIND BEST STRATEGIES ###


    i = 0
    while past_strategies != new_strategies:
        past_strategies = copy.copy(new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(grd.node[inode]['plays'])):
                grd.node[inode]['possible_play'] = grd.node[inode]['plays'][strategy]
                for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                    if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                grd.node[inode]['current_play'] = grd.node[inode]['plays'][random_index]
                new_strategies[inode] = grd.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        ind_payoff = 0
        for neighbor in xrange(len(grd.node[inode]['neighbors'])):
            if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                ind_payoff += 1
        ind_payoffs.append(ind_payoff)

    social_welfare = sum(ind_payoffs)

    ### OPTIMAL WELFARE APPROXIMATION ###

    source_neighborhoods = []

    # choose source with biggest ego_network
    for source in xrange(len(k)):
        source_neighborhoods.append(len(nx.ego_graph(gego, k[source], 2).nodes()))

    for node in xrange(len(nodeIDs)):
        gego.node[node]["neighbors"] = gdc[node]
        if node in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
            # every node in ego graph gets only one strategy (source's strategy)
            gego.node[node]["current_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["possible_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["plays"] = [k[np.argmax(source_neighborhoods)]]
        elif node not in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
        #    # prepare for best response
            gego.node[node]["current_play"] = node
            gego.node[node]["possible_play"] = node

    opt_past_strategies = [] # Initialize two lists to compare 
    opt_new_strategies = []
    for i in xrange(len(nodeIDs)):
        opt_new_strategies.append(gego.node[i]['current_play'])
        opt_past_strategies.append(0)


    ### While loop to find best strategies ###


    while opt_past_strategies != opt_new_strategies:
        opt_past_strategies = copy.copy(opt_new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(gego.node[inode]['plays'])):
                gego.node[inode]['possible_play'] = gego.node[inode]['plays'][strategy]
                for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                    if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                gego.node[inode]['current_play'] = gego.node[inode]['plays'][random_index]
                opt_new_strategies[inode] = gego.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    opt_ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        opt_ind_payoff = 0
        for neighbor in xrange(len(gego.node[inode]['neighbors'])):
            if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                opt_ind_payoff += 1
        opt_ind_payoffs.append(opt_ind_payoff)

    opt_social_welfare = sum(opt_ind_payoffs)

    if float(social_welfare) != 0:
        PoA = float(opt_social_welfare) / float(social_welfare)
    elif float(social_welfare) == 0:
        PoA = float('inf')

    ### 3
    # add data to dataframe k | k size | PoA
    df2 = pd.DataFrame([[k, len(k), social_welfare, opt_social_welfare, PoA ]], columns=('k', 'k size', 'SW', 'reasonable SW', 'PoA'))
    frames = [data, df2]
    data = pd.concat(frames)
data.to_csv('FB4-random-k%s.csv' %str(len(k)), encoding='utf-8')


# |K|=4

In [114]:
### EXPERIMENT K size = 4

sizes = [1,3,4,5,6,7,8,9,10]
# for j in xrange(len(sizes)): #for different sizes of k
    # initialize dataframe k | k size | PoA
data = pd.DataFrame(columns=('k', 'k size', 'PoA'))
    ### 1

    #random K set
    #k = random.sample(xrange(len(nodeIDs)), sizes[j])

for i in xrange(200):
    ### 1

    #random K set
    k = random.sample(xrange(len(nodeIDs)), sizes[2])

    ### 2

    #ADD ALL CALCULATIONS FOR THIS k
    grd = grdict.copy()
    gdc = copy.copy(grd)
    # make dictionary into graph
    grd = nx.to_networkx_graph(grd)

    ############## START STRATEGY DISTRIBUTION ###########

    ### Initialize Identity matrix - everyone has their own strategy

    strategies = np.identity(len(nodeIDs))

    ### Make a copy of real graph to find strategy distribution (because we'll need to delete some edges)

    H = grd.copy()

    ### For strategy distribution disregard edges between nodes in k

    #all possible edge combinations between the k_nodes
    knode_pedges=list(itertools.combinations(k, 2))

    #Identify if the nodes have edges among each other and store them
    edges_between_k=[]
    for pedge in knode_pedges:
        for edge in H.edges():
            if set(pedge)==set(edge):
                edges_between_k.append(pedge)

    # For the strategy distribution delete the edges between k in a duplicate H of our real graph grd! 
    H.remove_edges_from(edges_between_k)

    ### Find neighborhood of range = 2 for nodes in k and assign them the strategy of that node

    #Check how many nodes have neighborhood of 2 less than a 100

    for i in xrange(len(k)):
        for j in nx.ego_graph(H, k[i], 2).nodes():
            if j not in k:
                strategies[j][k[i]]=1

    np.where(strategies[4] == 1)[0].tolist()

    ### STRATEGY ASSIGNMENT TO NODES

    #to avoid confusion attribute will be called plays rather than strategies (to separate from strategy matrix)

    for i in xrange(len(nodeIDs)):
        key="plays"
        #step of assigining attributes
        grd.node[i].setdefault(key, np.where(strategies[i] == 1)[0].tolist()) 

    gego = grd.copy() # duplicate graph for 'OPTIMAL' calculations later

    ### Create ATTRIBUTES in graph: neighbors, current_play, possible_play (to find what is my payoff 
    #                                                                       if I play this strategy)

    for i in xrange(len(nodeIDs)):
        # step of assigining attributes
        grd.node[i].setdefault("neighbors", gdc[i])
        # everybody starts at their own strategy; 'possible_play' attribute for strategy update
        grd.node[i].setdefault("current_play", i)
        grd.node[i].setdefault("possible_play", i)

    ### UPDATE STRATEGIES ### 

    # the idea is to have two strategy lists that will be compared after every iteration through all the nodes
    # when the lists are identical, no player will switch strategies -> hence, Nash equilibrium


    # Initialize two lists to compare 
    past_strategies = []
    new_strategies = []
    for i in xrange(len(nodeIDs)):
        new_strategies.append(grd.node[i]['current_play'])
        past_strategies.append(0)


    ### WHILE LOOP TO FIND BEST STRATEGIES ###


    i = 0
    while past_strategies != new_strategies:
        past_strategies = copy.copy(new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(grd.node[inode]['plays'])):
                grd.node[inode]['possible_play'] = grd.node[inode]['plays'][strategy]
                for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                    if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                grd.node[inode]['current_play'] = grd.node[inode]['plays'][random_index]
                new_strategies[inode] = grd.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        ind_payoff = 0
        for neighbor in xrange(len(grd.node[inode]['neighbors'])):
            if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                ind_payoff += 1
        ind_payoffs.append(ind_payoff)

    social_welfare = sum(ind_payoffs)

    ### OPTIMAL WELFARE APPROXIMATION ###

    source_neighborhoods = []

    # choose source with biggest ego_network
    for source in xrange(len(k)):
        source_neighborhoods.append(len(nx.ego_graph(gego, k[source], 2).nodes()))

    for node in xrange(len(nodeIDs)):
        gego.node[node]["neighbors"] = gdc[node]
        if node in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
            # every node in ego graph gets only one strategy (source's strategy)
            gego.node[node]["current_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["possible_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["plays"] = [k[np.argmax(source_neighborhoods)]]
        elif node not in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
        #    # prepare for best response
            gego.node[node]["current_play"] = node
            gego.node[node]["possible_play"] = node

    opt_past_strategies = [] # Initialize two lists to compare 
    opt_new_strategies = []
    for i in xrange(len(nodeIDs)):
        opt_new_strategies.append(gego.node[i]['current_play'])
        opt_past_strategies.append(0)


    ### While loop to find best strategies ###


    while opt_past_strategies != opt_new_strategies:
        opt_past_strategies = copy.copy(opt_new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(gego.node[inode]['plays'])):
                gego.node[inode]['possible_play'] = gego.node[inode]['plays'][strategy]
                for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                    if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                gego.node[inode]['current_play'] = gego.node[inode]['plays'][random_index]
                opt_new_strategies[inode] = gego.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    opt_ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        opt_ind_payoff = 0
        for neighbor in xrange(len(gego.node[inode]['neighbors'])):
            if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                opt_ind_payoff += 1
        opt_ind_payoffs.append(opt_ind_payoff)

    opt_social_welfare = sum(opt_ind_payoffs)

    if float(social_welfare) != 0:
        PoA = float(opt_social_welfare) / float(social_welfare)
    elif float(social_welfare) == 0:
        PoA = float('inf')

    ### 3
    # add data to dataframe k | k size | PoA
    df2 = pd.DataFrame([[k, len(k), social_welfare, opt_social_welfare, PoA ]], columns=('k', 'k size', 'SW', 'reasonable SW', 'PoA'))
    frames = [data, df2]
    data = pd.concat(frames)
data.to_csv('FB4-random-k%s.csv' %str(len(k)), encoding='utf-8')


# |K|=5

In [115]:
### EXPERIMENT K size = 5

sizes = [1,3,4,5,6,7,8,9,10]
# for j in xrange(len(sizes)): #for different sizes of k
    # initialize dataframe k | k size | PoA
data = pd.DataFrame(columns=('k', 'k size', 'PoA'))
    ### 1

    #random K set
    #k = random.sample(xrange(len(nodeIDs)), sizes[j])

for i in xrange(200):
    ### 1

    #random K set
    k = random.sample(xrange(len(nodeIDs)), sizes[3])

    ### 2

    #ADD ALL CALCULATIONS FOR THIS k
    grd = grdict.copy()
    gdc = copy.copy(grd)
    # make dictionary into graph
    grd = nx.to_networkx_graph(grd)

    ############## START STRATEGY DISTRIBUTION ###########

    ### Initialize Identity matrix - everyone has their own strategy

    strategies = np.identity(len(nodeIDs))

    ### Make a copy of real graph to find strategy distribution (because we'll need to delete some edges)

    H = grd.copy()

    ### For strategy distribution disregard edges between nodes in k

    #all possible edge combinations between the k_nodes
    knode_pedges=list(itertools.combinations(k, 2))

    #Identify if the nodes have edges among each other and store them
    edges_between_k=[]
    for pedge in knode_pedges:
        for edge in H.edges():
            if set(pedge)==set(edge):
                edges_between_k.append(pedge)

    # For the strategy distribution delete the edges between k in a duplicate H of our real graph grd! 
    H.remove_edges_from(edges_between_k)

    ### Find neighborhood of range = 2 for nodes in k and assign them the strategy of that node

    #Check how many nodes have neighborhood of 2 less than a 100

    for i in xrange(len(k)):
        for j in nx.ego_graph(H, k[i], 2).nodes():
            if j not in k:
                strategies[j][k[i]]=1

    np.where(strategies[4] == 1)[0].tolist()

    ### STRATEGY ASSIGNMENT TO NODES

    #to avoid confusion attribute will be called plays rather than strategies (to separate from strategy matrix)

    for i in xrange(len(nodeIDs)):
        key="plays"
        #step of assigining attributes
        grd.node[i].setdefault(key, np.where(strategies[i] == 1)[0].tolist()) 

    gego = grd.copy() # duplicate graph for 'OPTIMAL' calculations later

    ### Create ATTRIBUTES in graph: neighbors, current_play, possible_play (to find what is my payoff 
    #                                                                       if I play this strategy)

    for i in xrange(len(nodeIDs)):
        # step of assigining attributes
        grd.node[i].setdefault("neighbors", gdc[i])
        # everybody starts at their own strategy; 'possible_play' attribute for strategy update
        grd.node[i].setdefault("current_play", i)
        grd.node[i].setdefault("possible_play", i)

    ### UPDATE STRATEGIES ### 

    # the idea is to have two strategy lists that will be compared after every iteration through all the nodes
    # when the lists are identical, no player will switch strategies -> hence, Nash equilibrium


    # Initialize two lists to compare 
    past_strategies = []
    new_strategies = []
    for i in xrange(len(nodeIDs)):
        new_strategies.append(grd.node[i]['current_play'])
        past_strategies.append(0)


    ### WHILE LOOP TO FIND BEST STRATEGIES ###


    i = 0
    while past_strategies != new_strategies:
        past_strategies = copy.copy(new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(grd.node[inode]['plays'])):
                grd.node[inode]['possible_play'] = grd.node[inode]['plays'][strategy]
                for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                    if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                grd.node[inode]['current_play'] = grd.node[inode]['plays'][random_index]
                new_strategies[inode] = grd.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        ind_payoff = 0
        for neighbor in xrange(len(grd.node[inode]['neighbors'])):
            if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                ind_payoff += 1
        ind_payoffs.append(ind_payoff)

    social_welfare = sum(ind_payoffs)

    ### OPTIMAL WELFARE APPROXIMATION ###

    source_neighborhoods = []

    # choose source with biggest ego_network
    for source in xrange(len(k)):
        source_neighborhoods.append(len(nx.ego_graph(gego, k[source], 2).nodes()))

    for node in xrange(len(nodeIDs)):
        gego.node[node]["neighbors"] = gdc[node]
        if node in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
            # every node in ego graph gets only one strategy (source's strategy)
            gego.node[node]["current_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["possible_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["plays"] = [k[np.argmax(source_neighborhoods)]]
        elif node not in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
        #    # prepare for best response
            gego.node[node]["current_play"] = node
            gego.node[node]["possible_play"] = node

    opt_past_strategies = [] # Initialize two lists to compare 
    opt_new_strategies = []
    for i in xrange(len(nodeIDs)):
        opt_new_strategies.append(gego.node[i]['current_play'])
        opt_past_strategies.append(0)


    ### While loop to find best strategies ###


    while opt_past_strategies != opt_new_strategies:
        opt_past_strategies = copy.copy(opt_new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(gego.node[inode]['plays'])):
                gego.node[inode]['possible_play'] = gego.node[inode]['plays'][strategy]
                for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                    if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                gego.node[inode]['current_play'] = gego.node[inode]['plays'][random_index]
                opt_new_strategies[inode] = gego.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    opt_ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        opt_ind_payoff = 0
        for neighbor in xrange(len(gego.node[inode]['neighbors'])):
            if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                opt_ind_payoff += 1
        opt_ind_payoffs.append(opt_ind_payoff)

    opt_social_welfare = sum(opt_ind_payoffs)

    if float(social_welfare) != 0:
        PoA = float(opt_social_welfare) / float(social_welfare)
    elif float(social_welfare) == 0:
        PoA = float('inf')

    ### 3
    # add data to dataframe k | k size | PoA
    df2 = pd.DataFrame([[k, len(k), social_welfare, opt_social_welfare, PoA ]], columns=('k', 'k size', 'SW', 'reasonable SW', 'PoA'))
    frames = [data, df2]
    data = pd.concat(frames)
data.to_csv('FB4-random-k%s.csv' %str(len(k)), encoding='utf-8')


# |K|=6

In [116]:
### EXPERIMENT K size = 6

sizes = [1,3,4,5,6,7,8,9,10]
# for j in xrange(len(sizes)): #for different sizes of k
    # initialize dataframe k | k size | PoA
data = pd.DataFrame(columns=('k', 'k size', 'PoA'))
    ### 1

    #random K set
    #k = random.sample(xrange(len(nodeIDs)), sizes[j])

for i in xrange(200):
    ### 1

    #random K set
    k = random.sample(xrange(len(nodeIDs)), sizes[4])

    ### 2

    #ADD ALL CALCULATIONS FOR THIS k
    grd = grdict.copy()
    gdc = copy.copy(grd)
    # make dictionary into graph
    grd = nx.to_networkx_graph(grd)

    ############## START STRATEGY DISTRIBUTION ###########

    ### Initialize Identity matrix - everyone has their own strategy

    strategies = np.identity(len(nodeIDs))

    ### Make a copy of real graph to find strategy distribution (because we'll need to delete some edges)

    H = grd.copy()

    ### For strategy distribution disregard edges between nodes in k

    #all possible edge combinations between the k_nodes
    knode_pedges=list(itertools.combinations(k, 2))

    #Identify if the nodes have edges among each other and store them
    edges_between_k=[]
    for pedge in knode_pedges:
        for edge in H.edges():
            if set(pedge)==set(edge):
                edges_between_k.append(pedge)

    # For the strategy distribution delete the edges between k in a duplicate H of our real graph grd! 
    H.remove_edges_from(edges_between_k)

    ### Find neighborhood of range = 2 for nodes in k and assign them the strategy of that node

    #Check how many nodes have neighborhood of 2 less than a 100

    for i in xrange(len(k)):
        for j in nx.ego_graph(H, k[i], 2).nodes():
            if j not in k:
                strategies[j][k[i]]=1

    np.where(strategies[4] == 1)[0].tolist()

    ### STRATEGY ASSIGNMENT TO NODES

    #to avoid confusion attribute will be called plays rather than strategies (to separate from strategy matrix)

    for i in xrange(len(nodeIDs)):
        key="plays"
        #step of assigining attributes
        grd.node[i].setdefault(key, np.where(strategies[i] == 1)[0].tolist()) 

    gego = grd.copy() # duplicate graph for 'OPTIMAL' calculations later

    ### Create ATTRIBUTES in graph: neighbors, current_play, possible_play (to find what is my payoff 
    #                                                                       if I play this strategy)

    for i in xrange(len(nodeIDs)):
        # step of assigining attributes
        grd.node[i].setdefault("neighbors", gdc[i])
        # everybody starts at their own strategy; 'possible_play' attribute for strategy update
        grd.node[i].setdefault("current_play", i)
        grd.node[i].setdefault("possible_play", i)

    ### UPDATE STRATEGIES ### 

    # the idea is to have two strategy lists that will be compared after every iteration through all the nodes
    # when the lists are identical, no player will switch strategies -> hence, Nash equilibrium


    # Initialize two lists to compare 
    past_strategies = []
    new_strategies = []
    for i in xrange(len(nodeIDs)):
        new_strategies.append(grd.node[i]['current_play'])
        past_strategies.append(0)


    ### WHILE LOOP TO FIND BEST STRATEGIES ###


    i = 0
    while past_strategies != new_strategies:
        past_strategies = copy.copy(new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(grd.node[inode]['plays'])):
                grd.node[inode]['possible_play'] = grd.node[inode]['plays'][strategy]
                for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                    if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                grd.node[inode]['current_play'] = grd.node[inode]['plays'][random_index]
                new_strategies[inode] = grd.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        ind_payoff = 0
        for neighbor in xrange(len(grd.node[inode]['neighbors'])):
            if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                ind_payoff += 1
        ind_payoffs.append(ind_payoff)

    social_welfare = sum(ind_payoffs)

    ### OPTIMAL WELFARE APPROXIMATION ###

    source_neighborhoods = []

    # choose source with biggest ego_network
    for source in xrange(len(k)):
        source_neighborhoods.append(len(nx.ego_graph(gego, k[source], 2).nodes()))

    for node in xrange(len(nodeIDs)):
        gego.node[node]["neighbors"] = gdc[node]
        if node in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
            # every node in ego graph gets only one strategy (source's strategy)
            gego.node[node]["current_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["possible_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["plays"] = [k[np.argmax(source_neighborhoods)]]
        elif node not in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
        #    # prepare for best response
            gego.node[node]["current_play"] = node
            gego.node[node]["possible_play"] = node

    opt_past_strategies = [] # Initialize two lists to compare 
    opt_new_strategies = []
    for i in xrange(len(nodeIDs)):
        opt_new_strategies.append(gego.node[i]['current_play'])
        opt_past_strategies.append(0)


    ### While loop to find best strategies ###


    while opt_past_strategies != opt_new_strategies:
        opt_past_strategies = copy.copy(opt_new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(gego.node[inode]['plays'])):
                gego.node[inode]['possible_play'] = gego.node[inode]['plays'][strategy]
                for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                    if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                gego.node[inode]['current_play'] = gego.node[inode]['plays'][random_index]
                opt_new_strategies[inode] = gego.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    opt_ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        opt_ind_payoff = 0
        for neighbor in xrange(len(gego.node[inode]['neighbors'])):
            if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                opt_ind_payoff += 1
        opt_ind_payoffs.append(opt_ind_payoff)

    opt_social_welfare = sum(opt_ind_payoffs)

    if float(social_welfare) != 0:
        PoA = float(opt_social_welfare) / float(social_welfare)
    elif float(social_welfare) == 0:
        PoA = float('inf')

    ### 3
    # add data to dataframe k | k size | PoA
    df2 = pd.DataFrame([[k, len(k), social_welfare, opt_social_welfare, PoA ]], columns=('k', 'k size', 'SW', 'reasonable SW', 'PoA'))
    frames = [data, df2]
    data = pd.concat(frames)
data.to_csv('FB4-random-k%s.csv' %str(len(k)), encoding='utf-8')


# |K| = 7

In [ ]:
### EXPERIMENT K size = 7

sizes = [1,3,4,5,6,7,8,9,10]
# for j in xrange(len(sizes)): #for different sizes of k
    # initialize dataframe k | k size | PoA
data = pd.DataFrame(columns=('k', 'k size', 'PoA'))
    ### 1

    #random K set
    #k = random.sample(xrange(len(nodeIDs)), sizes[j])

for i in xrange(200):
    ### 1

    #random K set
    k = random.sample(xrange(len(nodeIDs)), sizes[5])

    ### 2

    #ADD ALL CALCULATIONS FOR THIS k
    grd = grdict.copy()
    gdc = copy.copy(grd)
    # make dictionary into graph
    grd = nx.to_networkx_graph(grd)

    ############## START STRATEGY DISTRIBUTION ###########

    ### Initialize Identity matrix - everyone has their own strategy

    strategies = np.identity(len(nodeIDs))

    ### Make a copy of real graph to find strategy distribution (because we'll need to delete some edges)

    H = grd.copy()

    ### For strategy distribution disregard edges between nodes in k

    #all possible edge combinations between the k_nodes
    knode_pedges=list(itertools.combinations(k, 2))

    #Identify if the nodes have edges among each other and store them
    edges_between_k=[]
    for pedge in knode_pedges:
        for edge in H.edges():
            if set(pedge)==set(edge):
                edges_between_k.append(pedge)

    # For the strategy distribution delete the edges between k in a duplicate H of our real graph grd! 
    H.remove_edges_from(edges_between_k)

    ### Find neighborhood of range = 2 for nodes in k and assign them the strategy of that node

    #Check how many nodes have neighborhood of 2 less than a 100

    for i in xrange(len(k)):
        for j in nx.ego_graph(H, k[i], 2).nodes():
            if j not in k:
                strategies[j][k[i]]=1

    np.where(strategies[4] == 1)[0].tolist()

    ### STRATEGY ASSIGNMENT TO NODES

    #to avoid confusion attribute will be called plays rather than strategies (to separate from strategy matrix)

    for i in xrange(len(nodeIDs)):
        key="plays"
        #step of assigining attributes
        grd.node[i].setdefault(key, np.where(strategies[i] == 1)[0].tolist()) 

    gego = grd.copy() # duplicate graph for 'OPTIMAL' calculations later

    ### Create ATTRIBUTES in graph: neighbors, current_play, possible_play (to find what is my payoff 
    #                                                                       if I play this strategy)

    for i in xrange(len(nodeIDs)):
        # step of assigining attributes
        grd.node[i].setdefault("neighbors", gdc[i])
        # everybody starts at their own strategy; 'possible_play' attribute for strategy update
        grd.node[i].setdefault("current_play", i)
        grd.node[i].setdefault("possible_play", i)

    ### UPDATE STRATEGIES ### 

    # the idea is to have two strategy lists that will be compared after every iteration through all the nodes
    # when the lists are identical, no player will switch strategies -> hence, Nash equilibrium


    # Initialize two lists to compare 
    past_strategies = []
    new_strategies = []
    for i in xrange(len(nodeIDs)):
        new_strategies.append(grd.node[i]['current_play'])
        past_strategies.append(0)


    ### WHILE LOOP TO FIND BEST STRATEGIES ###


    i = 0
    while past_strategies != new_strategies:
        past_strategies = copy.copy(new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(grd.node[inode]['plays'])):
                grd.node[inode]['possible_play'] = grd.node[inode]['plays'][strategy]
                for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                    if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                grd.node[inode]['current_play'] = grd.node[inode]['plays'][random_index]
                new_strategies[inode] = grd.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        ind_payoff = 0
        for neighbor in xrange(len(grd.node[inode]['neighbors'])):
            if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                ind_payoff += 1
        ind_payoffs.append(ind_payoff)

    social_welfare = sum(ind_payoffs)

    ### OPTIMAL WELFARE APPROXIMATION ###

    source_neighborhoods = []

    # choose source with biggest ego_network
    for source in xrange(len(k)):
        source_neighborhoods.append(len(nx.ego_graph(gego, k[source], 2).nodes()))

    for node in xrange(len(nodeIDs)):
        gego.node[node]["neighbors"] = gdc[node]
        if node in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
            # every node in ego graph gets only one strategy (source's strategy)
            gego.node[node]["current_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["possible_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["plays"] = [k[np.argmax(source_neighborhoods)]]
        elif node not in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
        #    # prepare for best response
            gego.node[node]["current_play"] = node
            gego.node[node]["possible_play"] = node

    opt_past_strategies = [] # Initialize two lists to compare 
    opt_new_strategies = []
    for i in xrange(len(nodeIDs)):
        opt_new_strategies.append(gego.node[i]['current_play'])
        opt_past_strategies.append(0)


    ### While loop to find best strategies ###


    while opt_past_strategies != opt_new_strategies:
        opt_past_strategies = copy.copy(opt_new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(gego.node[inode]['plays'])):
                gego.node[inode]['possible_play'] = gego.node[inode]['plays'][strategy]
                for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                    if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                gego.node[inode]['current_play'] = gego.node[inode]['plays'][random_index]
                opt_new_strategies[inode] = gego.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    opt_ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        opt_ind_payoff = 0
        for neighbor in xrange(len(gego.node[inode]['neighbors'])):
            if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                opt_ind_payoff += 1
        opt_ind_payoffs.append(opt_ind_payoff)

    opt_social_welfare = sum(opt_ind_payoffs)

    if float(social_welfare) != 0:
        PoA = float(opt_social_welfare) / float(social_welfare)
    elif float(social_welfare) == 0:
        PoA = float('inf')

    ### 3
    # add data to dataframe k | k size | PoA
    df2 = pd.DataFrame([[k, len(k), social_welfare, opt_social_welfare, PoA ]], columns=('k', 'k size', 'SW', 'reasonable SW', 'PoA'))
    frames = [data, df2]
    data = pd.concat(frames)
data.to_csv('FB4-random-k%s.csv' %str(len(k)), encoding='utf-8')


# |K| = 8

In [ ]:
### EXPERIMENT K size = 8

sizes = [1,3,4,5,6,7,8,9,10]
# for j in xrange(len(sizes)): #for different sizes of k
    # initialize dataframe k | k size | PoA
data = pd.DataFrame(columns=('k', 'k size', 'PoA'))
    ### 1

    #random K set
    #k = random.sample(xrange(len(nodeIDs)), sizes[j])

for i in xrange(200):
    ### 1

    #random K set
    k = random.sample(xrange(len(nodeIDs)), sizes[6])

    ### 2

    #ADD ALL CALCULATIONS FOR THIS k
    grd = grdict.copy()
    gdc = copy.copy(grd)
    # make dictionary into graph
    grd = nx.to_networkx_graph(grd)

    ############## START STRATEGY DISTRIBUTION ###########

    ### Initialize Identity matrix - everyone has their own strategy

    strategies = np.identity(len(nodeIDs))

    ### Make a copy of real graph to find strategy distribution (because we'll need to delete some edges)

    H = grd.copy()

    ### For strategy distribution disregard edges between nodes in k

    #all possible edge combinations between the k_nodes
    knode_pedges=list(itertools.combinations(k, 2))

    #Identify if the nodes have edges among each other and store them
    edges_between_k=[]
    for pedge in knode_pedges:
        for edge in H.edges():
            if set(pedge)==set(edge):
                edges_between_k.append(pedge)

    # For the strategy distribution delete the edges between k in a duplicate H of our real graph grd! 
    H.remove_edges_from(edges_between_k)

    ### Find neighborhood of range = 2 for nodes in k and assign them the strategy of that node

    #Check how many nodes have neighborhood of 2 less than a 100

    for i in xrange(len(k)):
        for j in nx.ego_graph(H, k[i], 2).nodes():
            if j not in k:
                strategies[j][k[i]]=1

    np.where(strategies[4] == 1)[0].tolist()

    ### STRATEGY ASSIGNMENT TO NODES

    #to avoid confusion attribute will be called plays rather than strategies (to separate from strategy matrix)

    for i in xrange(len(nodeIDs)):
        key="plays"
        #step of assigining attributes
        grd.node[i].setdefault(key, np.where(strategies[i] == 1)[0].tolist()) 

    gego = grd.copy() # duplicate graph for 'OPTIMAL' calculations later

    ### Create ATTRIBUTES in graph: neighbors, current_play, possible_play (to find what is my payoff 
    #                                                                       if I play this strategy)

    for i in xrange(len(nodeIDs)):
        # step of assigining attributes
        grd.node[i].setdefault("neighbors", gdc[i])
        # everybody starts at their own strategy; 'possible_play' attribute for strategy update
        grd.node[i].setdefault("current_play", i)
        grd.node[i].setdefault("possible_play", i)

    ### UPDATE STRATEGIES ### 

    # the idea is to have two strategy lists that will be compared after every iteration through all the nodes
    # when the lists are identical, no player will switch strategies -> hence, Nash equilibrium


    # Initialize two lists to compare 
    past_strategies = []
    new_strategies = []
    for i in xrange(len(nodeIDs)):
        new_strategies.append(grd.node[i]['current_play'])
        past_strategies.append(0)


    ### WHILE LOOP TO FIND BEST STRATEGIES ###


    i = 0
    while past_strategies != new_strategies:
        past_strategies = copy.copy(new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(grd.node[inode]['plays'])):
                grd.node[inode]['possible_play'] = grd.node[inode]['plays'][strategy]
                for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                    if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                grd.node[inode]['current_play'] = grd.node[inode]['plays'][random_index]
                new_strategies[inode] = grd.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        ind_payoff = 0
        for neighbor in xrange(len(grd.node[inode]['neighbors'])):
            if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                ind_payoff += 1
        ind_payoffs.append(ind_payoff)

    social_welfare = sum(ind_payoffs)

    ### OPTIMAL WELFARE APPROXIMATION ###

    source_neighborhoods = []

    # choose source with biggest ego_network
    for source in xrange(len(k)):
        source_neighborhoods.append(len(nx.ego_graph(gego, k[source], 2).nodes()))

    for node in xrange(len(nodeIDs)):
        gego.node[node]["neighbors"] = gdc[node]
        if node in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
            # every node in ego graph gets only one strategy (source's strategy)
            gego.node[node]["current_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["possible_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["plays"] = [k[np.argmax(source_neighborhoods)]]
        elif node not in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
        #    # prepare for best response
            gego.node[node]["current_play"] = node
            gego.node[node]["possible_play"] = node

    opt_past_strategies = [] # Initialize two lists to compare 
    opt_new_strategies = []
    for i in xrange(len(nodeIDs)):
        opt_new_strategies.append(gego.node[i]['current_play'])
        opt_past_strategies.append(0)


    ### While loop to find best strategies ###


    while opt_past_strategies != opt_new_strategies:
        opt_past_strategies = copy.copy(opt_new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(gego.node[inode]['plays'])):
                gego.node[inode]['possible_play'] = gego.node[inode]['plays'][strategy]
                for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                    if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                gego.node[inode]['current_play'] = gego.node[inode]['plays'][random_index]
                opt_new_strategies[inode] = gego.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    opt_ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        opt_ind_payoff = 0
        for neighbor in xrange(len(gego.node[inode]['neighbors'])):
            if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                opt_ind_payoff += 1
        opt_ind_payoffs.append(opt_ind_payoff)

    opt_social_welfare = sum(opt_ind_payoffs)

    if float(social_welfare) != 0:
        PoA = float(opt_social_welfare) / float(social_welfare)
    elif float(social_welfare) == 0:
        PoA = float('inf')

    ### 3
    # add data to dataframe k | k size | PoA
    df2 = pd.DataFrame([[k, len(k), social_welfare, opt_social_welfare, PoA ]], columns=('k', 'k size', 'SW', 'reasonable SW', 'PoA'))
    frames = [data, df2]
    data = pd.concat(frames)
data.to_csv('FB4-random-k%s.csv' %str(len(k)), encoding='utf-8')


# |K| = 9

In [ ]:
### EXPERIMENT K size = 9

sizes = [1,3,4,5,6,7,8,9,10]
# for j in xrange(len(sizes)): #for different sizes of k
    # initialize dataframe k | k size | PoA
data = pd.DataFrame(columns=('k', 'k size', 'PoA'))
    ### 1

    #random K set
    #k = random.sample(xrange(len(nodeIDs)), sizes[j])

for i in xrange(200):
    ### 1

    #random K set
    k = random.sample(xrange(len(nodeIDs)), sizes[7])

    ### 2

    #ADD ALL CALCULATIONS FOR THIS k
    grd = grdict.copy()
    gdc = copy.copy(grd)
    # make dictionary into graph
    grd = nx.to_networkx_graph(grd)

    ############## START STRATEGY DISTRIBUTION ###########

    ### Initialize Identity matrix - everyone has their own strategy

    strategies = np.identity(len(nodeIDs))

    ### Make a copy of real graph to find strategy distribution (because we'll need to delete some edges)

    H = grd.copy()

    ### For strategy distribution disregard edges between nodes in k

    #all possible edge combinations between the k_nodes
    knode_pedges=list(itertools.combinations(k, 2))

    #Identify if the nodes have edges among each other and store them
    edges_between_k=[]
    for pedge in knode_pedges:
        for edge in H.edges():
            if set(pedge)==set(edge):
                edges_between_k.append(pedge)

    # For the strategy distribution delete the edges between k in a duplicate H of our real graph grd! 
    H.remove_edges_from(edges_between_k)

    ### Find neighborhood of range = 2 for nodes in k and assign them the strategy of that node

    #Check how many nodes have neighborhood of 2 less than a 100

    for i in xrange(len(k)):
        for j in nx.ego_graph(H, k[i], 2).nodes():
            if j not in k:
                strategies[j][k[i]]=1

    np.where(strategies[4] == 1)[0].tolist()

    ### STRATEGY ASSIGNMENT TO NODES

    #to avoid confusion attribute will be called plays rather than strategies (to separate from strategy matrix)

    for i in xrange(len(nodeIDs)):
        key="plays"
        #step of assigining attributes
        grd.node[i].setdefault(key, np.where(strategies[i] == 1)[0].tolist()) 

    gego = grd.copy() # duplicate graph for 'OPTIMAL' calculations later

    ### Create ATTRIBUTES in graph: neighbors, current_play, possible_play (to find what is my payoff 
    #                                                                       if I play this strategy)

    for i in xrange(len(nodeIDs)):
        # step of assigining attributes
        grd.node[i].setdefault("neighbors", gdc[i])
        # everybody starts at their own strategy; 'possible_play' attribute for strategy update
        grd.node[i].setdefault("current_play", i)
        grd.node[i].setdefault("possible_play", i)

    ### UPDATE STRATEGIES ### 

    # the idea is to have two strategy lists that will be compared after every iteration through all the nodes
    # when the lists are identical, no player will switch strategies -> hence, Nash equilibrium


    # Initialize two lists to compare 
    past_strategies = []
    new_strategies = []
    for i in xrange(len(nodeIDs)):
        new_strategies.append(grd.node[i]['current_play'])
        past_strategies.append(0)


    ### WHILE LOOP TO FIND BEST STRATEGIES ###


    i = 0
    while past_strategies != new_strategies:
        past_strategies = copy.copy(new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(grd.node[inode]['plays'])):
                grd.node[inode]['possible_play'] = grd.node[inode]['plays'][strategy]
                for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                    if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                grd.node[inode]['current_play'] = grd.node[inode]['plays'][random_index]
                new_strategies[inode] = grd.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        ind_payoff = 0
        for neighbor in xrange(len(grd.node[inode]['neighbors'])):
            if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                ind_payoff += 1
        ind_payoffs.append(ind_payoff)

    social_welfare = sum(ind_payoffs)

    ### OPTIMAL WELFARE APPROXIMATION ###

    source_neighborhoods = []

    # choose source with biggest ego_network
    for source in xrange(len(k)):
        source_neighborhoods.append(len(nx.ego_graph(gego, k[source], 2).nodes()))

    for node in xrange(len(nodeIDs)):
        gego.node[node]["neighbors"] = gdc[node]
        if node in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
            # every node in ego graph gets only one strategy (source's strategy)
            gego.node[node]["current_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["possible_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["plays"] = [k[np.argmax(source_neighborhoods)]]
        elif node not in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
        #    # prepare for best response
            gego.node[node]["current_play"] = node
            gego.node[node]["possible_play"] = node

    opt_past_strategies = [] # Initialize two lists to compare 
    opt_new_strategies = []
    for i in xrange(len(nodeIDs)):
        opt_new_strategies.append(gego.node[i]['current_play'])
        opt_past_strategies.append(0)


    ### While loop to find best strategies ###


    while opt_past_strategies != opt_new_strategies:
        opt_past_strategies = copy.copy(opt_new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(gego.node[inode]['plays'])):
                gego.node[inode]['possible_play'] = gego.node[inode]['plays'][strategy]
                for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                    if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                gego.node[inode]['current_play'] = gego.node[inode]['plays'][random_index]
                opt_new_strategies[inode] = gego.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    opt_ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        opt_ind_payoff = 0
        for neighbor in xrange(len(gego.node[inode]['neighbors'])):
            if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                opt_ind_payoff += 1
        opt_ind_payoffs.append(opt_ind_payoff)

    opt_social_welfare = sum(opt_ind_payoffs)

    if float(social_welfare) != 0:
        PoA = float(opt_social_welfare) / float(social_welfare)
    elif float(social_welfare) == 0:
        PoA = float('inf')

    ### 3
    # add data to dataframe k | k size | PoA
    df2 = pd.DataFrame([[k, len(k), social_welfare, opt_social_welfare, PoA ]], columns=('k', 'k size', 'SW', 'reasonable SW', 'PoA'))
    frames = [data, df2]
    data = pd.concat(frames)
data.to_csv('FB4-random-k%s.csv' %str(len(k)), encoding='utf-8')


# |K| = 10

In [ ]:
### EXPERIMENT K size = 10

sizes = [1,3,4,5,6,7,8,9,10]
# for j in xrange(len(sizes)): #for different sizes of k
    # initialize dataframe k | k size | PoA
data = pd.DataFrame(columns=('k', 'k size', 'PoA'))
    ### 1

    #random K set
    #k = random.sample(xrange(len(nodeIDs)), sizes[j])

for i in xrange(200):
    ### 1

    #random K set
    k = random.sample(xrange(len(nodeIDs)), sizes[8])

    ### 2

    #ADD ALL CALCULATIONS FOR THIS k
    grd = grdict.copy()
    gdc = copy.copy(grd)
    # make dictionary into graph
    grd = nx.to_networkx_graph(grd)

    ############## START STRATEGY DISTRIBUTION ###########

    ### Initialize Identity matrix - everyone has their own strategy

    strategies = np.identity(len(nodeIDs))

    ### Make a copy of real graph to find strategy distribution (because we'll need to delete some edges)

    H = grd.copy()

    ### For strategy distribution disregard edges between nodes in k

    #all possible edge combinations between the k_nodes
    knode_pedges=list(itertools.combinations(k, 2))

    #Identify if the nodes have edges among each other and store them
    edges_between_k=[]
    for pedge in knode_pedges:
        for edge in H.edges():
            if set(pedge)==set(edge):
                edges_between_k.append(pedge)

    # For the strategy distribution delete the edges between k in a duplicate H of our real graph grd! 
    H.remove_edges_from(edges_between_k)

    ### Find neighborhood of range = 2 for nodes in k and assign them the strategy of that node

    #Check how many nodes have neighborhood of 2 less than a 100

    for i in xrange(len(k)):
        for j in nx.ego_graph(H, k[i], 2).nodes():
            if j not in k:
                strategies[j][k[i]]=1

    np.where(strategies[4] == 1)[0].tolist()

    ### STRATEGY ASSIGNMENT TO NODES

    #to avoid confusion attribute will be called plays rather than strategies (to separate from strategy matrix)

    for i in xrange(len(nodeIDs)):
        key="plays"
        #step of assigining attributes
        grd.node[i].setdefault(key, np.where(strategies[i] == 1)[0].tolist()) 

    gego = grd.copy() # duplicate graph for 'OPTIMAL' calculations later

    ### Create ATTRIBUTES in graph: neighbors, current_play, possible_play (to find what is my payoff 
    #                                                                       if I play this strategy)

    for i in xrange(len(nodeIDs)):
        # step of assigining attributes
        grd.node[i].setdefault("neighbors", gdc[i])
        # everybody starts at their own strategy; 'possible_play' attribute for strategy update
        grd.node[i].setdefault("current_play", i)
        grd.node[i].setdefault("possible_play", i)

    ### UPDATE STRATEGIES ### 

    # the idea is to have two strategy lists that will be compared after every iteration through all the nodes
    # when the lists are identical, no player will switch strategies -> hence, Nash equilibrium


    # Initialize two lists to compare 
    past_strategies = []
    new_strategies = []
    for i in xrange(len(nodeIDs)):
        new_strategies.append(grd.node[i]['current_play'])
        past_strategies.append(0)


    ### WHILE LOOP TO FIND BEST STRATEGIES ###


    i = 0
    while past_strategies != new_strategies:
        past_strategies = copy.copy(new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(grd.node[inode]['plays'])):
                grd.node[inode]['possible_play'] = grd.node[inode]['plays'][strategy]
                for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                    if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                grd.node[inode]['current_play'] = grd.node[inode]['plays'][random_index]
                new_strategies[inode] = grd.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        ind_payoff = 0
        for neighbor in xrange(len(grd.node[inode]['neighbors'])):
            if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                ind_payoff += 1
        ind_payoffs.append(ind_payoff)

    social_welfare = sum(ind_payoffs)

    ### OPTIMAL WELFARE APPROXIMATION ###

    source_neighborhoods = []

    # choose source with biggest ego_network
    for source in xrange(len(k)):
        source_neighborhoods.append(len(nx.ego_graph(gego, k[source], 2).nodes()))

    for node in xrange(len(nodeIDs)):
        gego.node[node]["neighbors"] = gdc[node]
        if node in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
            # every node in ego graph gets only one strategy (source's strategy)
            gego.node[node]["current_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["possible_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["plays"] = [k[np.argmax(source_neighborhoods)]]
        elif node not in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
        #    # prepare for best response
            gego.node[node]["current_play"] = node
            gego.node[node]["possible_play"] = node

    opt_past_strategies = [] # Initialize two lists to compare 
    opt_new_strategies = []
    for i in xrange(len(nodeIDs)):
        opt_new_strategies.append(gego.node[i]['current_play'])
        opt_past_strategies.append(0)


    ### While loop to find best strategies ###


    while opt_past_strategies != opt_new_strategies:
        opt_past_strategies = copy.copy(opt_new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(gego.node[inode]['plays'])):
                gego.node[inode]['possible_play'] = gego.node[inode]['plays'][strategy]
                for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                    if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                gego.node[inode]['current_play'] = gego.node[inode]['plays'][random_index]
                opt_new_strategies[inode] = gego.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    opt_ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        opt_ind_payoff = 0
        for neighbor in xrange(len(gego.node[inode]['neighbors'])):
            if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                opt_ind_payoff += 1
        opt_ind_payoffs.append(opt_ind_payoff)

    opt_social_welfare = sum(opt_ind_payoffs)

    if float(social_welfare) != 0:
        PoA = float(opt_social_welfare) / float(social_welfare)
    elif float(social_welfare) == 0:
        PoA = float('inf')

    ### 3
    # add data to dataframe k | k size | PoA
    df2 = pd.DataFrame([[k, len(k), social_welfare, opt_social_welfare, PoA ]], columns=('k', 'k size', 'SW', 'reasonable SW', 'PoA'))
    frames = [data, df2]
    data = pd.concat(frames)
data.to_csv('FB4-random-k%s.csv' %str(len(k)), encoding='utf-8')


# |K| = 2

In [ ]:
### EXPERIMENT K size = 2

sizes = [1,3,4,5,6,7,8,9,10,2]
# for j in xrange(len(sizes)): #for different sizes of k
    # initialize dataframe k | k size | PoA
data = pd.DataFrame(columns=('k', 'k size', 'PoA'))
    ### 1

    #random K set
    #k = random.sample(xrange(len(nodeIDs)), sizes[j])

for i in xrange(200):
    ### 1

    #random K set
    k = random.sample(xrange(len(nodeIDs)), sizes[9])

    ### 2

    #ADD ALL CALCULATIONS FOR THIS k
    grd = grdict.copy()
    gdc = copy.copy(grd)
    # make dictionary into graph
    grd = nx.to_networkx_graph(grd)

    ############## START STRATEGY DISTRIBUTION ###########

    ### Initialize Identity matrix - everyone has their own strategy

    strategies = np.identity(len(nodeIDs))

    ### Make a copy of real graph to find strategy distribution (because we'll need to delete some edges)

    H = grd.copy()

    ### For strategy distribution disregard edges between nodes in k

    #all possible edge combinations between the k_nodes
    knode_pedges=list(itertools.combinations(k, 2))

    #Identify if the nodes have edges among each other and store them
    edges_between_k=[]
    for pedge in knode_pedges:
        for edge in H.edges():
            if set(pedge)==set(edge):
                edges_between_k.append(pedge)

    # For the strategy distribution delete the edges between k in a duplicate H of our real graph grd! 
    H.remove_edges_from(edges_between_k)

    ### Find neighborhood of range = 2 for nodes in k and assign them the strategy of that node

    #Check how many nodes have neighborhood of 2 less than a 100

    for i in xrange(len(k)):
        for j in nx.ego_graph(H, k[i], 2).nodes():
            if j not in k:
                strategies[j][k[i]]=1

    np.where(strategies[4] == 1)[0].tolist()

    ### STRATEGY ASSIGNMENT TO NODES

    #to avoid confusion attribute will be called plays rather than strategies (to separate from strategy matrix)

    for i in xrange(len(nodeIDs)):
        key="plays"
        #step of assigining attributes
        grd.node[i].setdefault(key, np.where(strategies[i] == 1)[0].tolist()) 

    gego = grd.copy() # duplicate graph for 'OPTIMAL' calculations later

    ### Create ATTRIBUTES in graph: neighbors, current_play, possible_play (to find what is my payoff 
    #                                                                       if I play this strategy)

    for i in xrange(len(nodeIDs)):
        # step of assigining attributes
        grd.node[i].setdefault("neighbors", gdc[i])
        # everybody starts at their own strategy; 'possible_play' attribute for strategy update
        grd.node[i].setdefault("current_play", i)
        grd.node[i].setdefault("possible_play", i)

    ### UPDATE STRATEGIES ### 

    # the idea is to have two strategy lists that will be compared after every iteration through all the nodes
    # when the lists are identical, no player will switch strategies -> hence, Nash equilibrium


    # Initialize two lists to compare 
    past_strategies = []
    new_strategies = []
    for i in xrange(len(nodeIDs)):
        new_strategies.append(grd.node[i]['current_play'])
        past_strategies.append(0)


    ### WHILE LOOP TO FIND BEST STRATEGIES ###


    i = 0
    while past_strategies != new_strategies:
        past_strategies = copy.copy(new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(grd.node[inode]['plays'])):
                grd.node[inode]['possible_play'] = grd.node[inode]['plays'][strategy]
                for neighbor in xrange(len(grd.node[inode]['neighbors'])):
                    if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                grd.node[inode]['current_play'] = grd.node[inode]['plays'][random_index]
                new_strategies[inode] = grd.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        ind_payoff = 0
        for neighbor in xrange(len(grd.node[inode]['neighbors'])):
            if grd.node[grd.node[inode]['neighbors'][neighbor]]['current_play'] == grd.node[inode]['current_play']:
                ind_payoff += 1
        ind_payoffs.append(ind_payoff)

    social_welfare = sum(ind_payoffs)

    ### OPTIMAL WELFARE APPROXIMATION ###

    source_neighborhoods = []

    # choose source with biggest ego_network
    for source in xrange(len(k)):
        source_neighborhoods.append(len(nx.ego_graph(gego, k[source], 2).nodes()))

    for node in xrange(len(nodeIDs)):
        gego.node[node]["neighbors"] = gdc[node]
        if node in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
            # every node in ego graph gets only one strategy (source's strategy)
            gego.node[node]["current_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["possible_play"] = k[np.argmax(source_neighborhoods)]
            gego.node[node]["plays"] = [k[np.argmax(source_neighborhoods)]]
        elif node not in nx.ego_graph(gego, np.max(source_neighborhoods), 2).nodes():
        #    # prepare for best response
            gego.node[node]["current_play"] = node
            gego.node[node]["possible_play"] = node

    opt_past_strategies = [] # Initialize two lists to compare 
    opt_new_strategies = []
    for i in xrange(len(nodeIDs)):
        opt_new_strategies.append(gego.node[i]['current_play'])
        opt_past_strategies.append(0)


    ### While loop to find best strategies ###


    while opt_past_strategies != opt_new_strategies:
        opt_past_strategies = copy.copy(opt_new_strategies)
        for inode in xrange(len(nodeIDs)):
            # current payoff initialization
            curr_payoff = 0
            # possible payoff/list of payoffs initialize for every node
            pp = 0
            pps = []

            # CURRENT payoff
            for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                    curr_payoff += 1

            # POSSIBLE payoff
            for strategy in xrange(len(gego.node[inode]['plays'])):
                gego.node[inode]['possible_play'] = gego.node[inode]['plays'][strategy]
                for neighbor in xrange(len(gego.node[inode]['neighbors'])):
                    if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['possible_play']:
                         pp += 1
                pps.append(pp)
                pp = 0

            # Find strategy giving best payoff and update 'current_play' according to it
            if np.max(pps) > curr_payoff:
                # randomize strategy if max payoffs repeat
                max_payoff_indices = [i for i, x in enumerate(pps) if x == max(pps)]
                random_index = random.choice(max_payoff_indices)
                # assign new strategy
                gego.node[inode]['current_play'] = gego.node[inode]['plays'][random_index]
                opt_new_strategies[inode] = gego.node[inode]['current_play']

    ### CALCULATE SOCIAL WELFARE ###


    # initialize list with all players' payoffs
    opt_ind_payoffs = []
    for inode in xrange(len(nodeIDs)):
        opt_ind_payoff = 0
        for neighbor in xrange(len(gego.node[inode]['neighbors'])):
            if gego.node[gego.node[inode]['neighbors'][neighbor]]['current_play'] == gego.node[inode]['current_play']:
                opt_ind_payoff += 1
        opt_ind_payoffs.append(opt_ind_payoff)

    opt_social_welfare = sum(opt_ind_payoffs)

    if float(social_welfare) != 0:
        PoA = float(opt_social_welfare) / float(social_welfare)
    elif float(social_welfare) == 0:
        PoA = float('inf')

    ### 3
    # add data to dataframe k | k size | PoA
    df2 = pd.DataFrame([[k, len(k), social_welfare, opt_social_welfare, PoA ]], columns=('k', 'k size', 'SW', 'reasonable SW', 'PoA'))
    frames = [data, df2]
    data = pd.concat(frames)
data.to_csv('FB4-random-k%s.csv' %str(len(k)), encoding='utf-8')
